In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor,IsolationForest
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from category_encoders import TargetEncoder
from sklearn.impute import IterativeImputer
import sys
import os 
from itertools import product, combinations
pd.options.display.float_format = "{:,.2f}".format 
pd.set_option("display.max_columns", None)
sys.path.append(os.path.abspath("../../src"))   
import soporte_preprocesamiento as f
import soporte_regresion as r

ImportError: IterativeImputer is experimental and the API might change without any deprecation cycle. To use it, you need to explicitly import enable_iterative_imputer:
from sklearn.experimental import enable_iterative_imputer

In [30]:

data = pd.read_csv("../datos/datos_sin_nulos.csv", index_col=0).reset_index(drop=True)



In [31]:
data.head()

,price,propertyType,size,exterior,rooms,bathrooms,province,municipality,distance,status,floor,district,hasLift,parkingSpace
0,750.00,flat,60.00,True,1,1,Madrid,Madrid,7037,good,3,Hortaleza,True,desconocido
1,750.00,flat,70.00,True,2,1,Madrid,San Sebastián de los Reyes,16145,good,bj,Centro Urbano,False,desconocido
2,400.00,penthouse,67.00,True,2,2,Madrid,Villamanrique de Tajo,55041,good,2,desconocido,False,desconocido
3,450.00,flat,89.00,False,2,1,Afueras,Recas,47186,good,2,desconocido,True,True
4,450.00,flat,60.00,True,2,1,Afueras,Cedillo del Condado,38237,good,1,desconocido,False,desconocido


In [32]:
x = data.drop(columns=['price'])
y = data['price']

# Preprocesado

In [33]:
x["exterior"]=x["exterior"].astype(str)
x["rooms"]=x["rooms"].astype(str)
x["bathrooms"]=x["bathrooms"].astype(str)

In [34]:
diccionario_encoding = {"onehot": ["propertyType", "exterior", "rooms", "status", "floor", "hasLift", "parkingSpace"], 
                        "dummies": [], # no metemos ninguna
                        'ordinal' : { }, #no metemos ninguna
                        "label": [] , # no metemos ninguna columna porque no queremos en ningún caso que se asignen las categorías de forma aleatoria
                        "frequency": [], # no metemos ninguna columna porque no coincide el orden del value counts con las categorias y la variable respuesta
                        "target": ["bathrooms", "province", "municipality", "district"]  
                        }

In [35]:
# Hacemos la gestion de las variables categoricas

one_hot_encoder = OneHotEncoder()
col_encode = diccionario_encoding.get("onehot", [])
trans_one_hot = one_hot_encoder.fit_transform(x[col_encode])
oh_df = pd.DataFrame(trans_one_hot.toarray(), columns=one_hot_encoder.get_feature_names_out())

# concatenamos los resultados obtenidos en la transformación con el DataFrame original
x = pd.concat([x.reset_index(drop=True), oh_df.reset_index(drop=True)], axis=1)
lista_eliminar=["propertyType", "exterior", "rooms", "status", "floor", "hasLift", "parkingSpace"]
x.drop(columns=lista_eliminar, inplace=True)
x_onehoteado=x

In [36]:
categorical_columns = x.select_dtypes(include='O').columns
numerical_columns = x.select_dtypes(include=np.number).columns

In [37]:
target_encoder = TargetEncoder(cols=categorical_columns)
x_encoded = target_encoder.fit_transform(x_onehoteado, y)


In [38]:

# Escalamos las variables

scaler = MinMaxScaler()
x_encoded[numerical_columns] = scaler.fit_transform(x_encoded[numerical_columns])

In [39]:


# Guardar los transformadores (TargetEncoder y StandardScaler)
with open('target_encoder.pkl', 'wb') as f:
    pickle.dump(target_encoder, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [41]:
x_encoded.head()

,size,bathrooms,province,municipality,distance,district,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,exterior_False,exterior_True,rooms_0,rooms_1,rooms_2,rooms_3,rooms_4,status_desconocido,status_good,status_newdevelopment,status_renew,floor_1,floor_14,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_bj,floor_desconocido,floor_en,floor_ss,floor_st,hasLift_False,hasLift_True,hasLift_desconocido,parkingSpace_False,parkingSpace_True,parkingSpace_desconocido
0,0.31,677.48,693.41,705.59,0.11,684.90,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00
1,0.39,677.48,693.41,685.07,0.27,683.25,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
2,0.36,646.96,693.41,637.09,0.92,592.06,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
3,0.53,677.48,582.04,643.59,0.79,592.06,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00
4,0.31,677.48,582.04,648.07,0.64,592.06,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00


Ya q el dataframe del ifo es x no he podido meter "price" pero dado que es la variable a predecir casi es hatsa positivo

In [42]:
# Aquí tratamos los outliers


contaminacion= [0.01, 0.05, 0.1]
estimadores = [100,400,1000, 2000] 
combinaciones= list(product(contaminacion, estimadores))
for cont, esti in combinaciones:
    
    ifo=IsolationForest(random_state=42, n_estimators=esti, contamination= cont, n_jobs=-1)         #n_estimator es el número de árboles y n_jobs con -1 coge todos los nucleos del ordenador

    x_encoded[f"outliers_ifo_{cont}_{esti}"]=ifo.fit_predict(x_encoded[["size",	"bathrooms", "province",	"municipality",	"distance",	"district",	"propertyType_chalet",
        "propertyType_countryHouse",	"propertyType_duplex",	"propertyType_flat","propertyType_penthouse",	"propertyType_studio"	,"exterior_False",	"exterior_True","rooms_0",
        "rooms_1", "rooms_2",	"rooms_3",	"rooms_4",	"status_desconocido",	"status_good",	"status_newdevelopment",	"status_renew",	"floor_1",	"floor_14",	"floor_2",	"floor_3",
    	"floor_4",	"floor_5",	"floor_6",	"floor_7", "floor_8",	"floor_bj",	"floor_desconocido",	"floor_en",	"floor_ss",	"floor_st",	"hasLift_False",	"hasLift_True",
    	"hasLift_desconocido", "parkingSpace_False", "parkingSpace_True", "parkingSpace_desconocido"]]) 


In [43]:
columnasdf=x_encoded.filter(like="outliers_ifo")
columnas_ifo=columnasdf.columns

In [44]:

filtered_df = x_encoded[(x_encoded[columnas_ifo] == -1).all(axis=1)]

Al observar las filas que tienen todos los valores con -1 y observar que suponen una parte ínfima de los datos, se decide eliminarlos

In [45]:
df_result = x_encoded.drop(index = filtered_df.index)
df_result.reset_index(drop=True, inplace=True)
proporcion = 0.6 * len(columnas_ifo)
df_outliers_60 = df_result[df_result[columnas_ifo].eq(-1).sum(axis=1) >= proporcion]

In [48]:
df_outliers_60=df_outliers_60[(df_outliers_60["distance"] > 0 ) & (df_outliers_60["distance"] < 0.9)]
df_outliers_60=df_outliers_60[(df_outliers_60["size"] > 0.35 ) & (df_outliers_60["size"] < 0.84)]

In [49]:
df_result.loc[df_outliers_60.index, "size"] = np.nan
df_result.loc[df_outliers_60.index, "distance"] = np.nan

In [50]:
df_numericas=df_result.select_dtypes(include=np.number)
imputer_iterative= IterativeImputer()       #Aquí se puede poner el KNNImputer o el RandomForestRegressor
                                            #missing values es por si los nulos no estan como np.nan sino en texto por ejemplo (aunque lo suyo es limpiarlo en el EDA)
                                            #initial_strategy= "median" sirve para indicar que me lo haga con las medianas
iterative_imputado=imputer_iterative.fit_transform(df_numericas)   #como es solo para numericas se tiene que meter el df de numericas pero hay que definirlo otra vez porque hemos dropeado duplicados
df_iterativo=df_result.copy()
df_num_sin_nulos=pd.DataFrame(iterative_imputado, columns= df_numericas.columns)  #para que tenga los mismo nombres de columnas
df_iterativo[df_numericas.columns]= df_num_sin_nulos    #Con esto metemos todo el dataframe de las numericas sin nulos donde corresponda rellenando esos nulos

In [ ]:
lista_eliminar=['outliers_ifo_0.01_100',
       'outliers_ifo_0.01_400', 'outliers_ifo_0.01_1000',
       'outliers_ifo_0.01_2000', 'outliers_ifo_0.05_100',
       'outliers_ifo_0.05_400', 'outliers_ifo_0.05_1000',
       'outliers_ifo_0.05_2000', 'outliers_ifo_0.1_100',
       'outliers_ifo_0.1_400', 'outliers_ifo_0.1_1000',
       'outliers_ifo_0.1_2000']
df_iterativo.drop(columns= lista_eliminar, inplace=True)
x_sinout=df_iterativo

# Modelo

In [68]:
x_sinout.columns

Index(['size', 'bathrooms', 'province', 'municipality', 'distance', 'district',
       'propertyType_chalet', 'propertyType_countryHouse',
       'propertyType_duplex', 'propertyType_flat', 'propertyType_penthouse',
       'propertyType_studio', 'exterior_False', 'exterior_True', 'rooms_0',
       'rooms_1', 'rooms_2', 'rooms_3', 'rooms_4', 'status_desconocido',
       'status_good', 'status_newdevelopment', 'status_renew', 'floor_1',
       'floor_14', 'floor_2', 'floor_3', 'floor_4', 'floor_5', 'floor_6',
       'floor_7', 'floor_8', 'floor_bj', 'floor_desconocido', 'floor_en',
       'floor_ss', 'floor_st', 'hasLift_False', 'hasLift_True',
       'hasLift_desconocido', 'parkingSpace_False', 'parkingSpace_True',
       'parkingSpace_desconocido'],
      dtype='object')

In [69]:
lista_encabezados=['size', 'bathrooms', 'province', 'municipality', 'distance', 'district',
       'propertyType_chalet', 'propertyType_countryHouse',
       'propertyType_duplex', 'propertyType_flat', 'propertyType_penthouse',
       'propertyType_studio', 'exterior_False', 'exterior_True', 'rooms_0',
       'rooms_1', 'rooms_2', 'rooms_3', 'rooms_4', 'status_desconocido',
       'status_good', 'status_newdevelopment', 'status_renew', 'floor_1',
       'floor_14', 'floor_2', 'floor_3', 'floor_4', 'floor_5', 'floor_6',
       'floor_7', 'floor_8', 'floor_bj', 'floor_desconocido', 'floor_en',
       'floor_ss', 'floor_st', 'hasLift_False', 'hasLift_True',
       'hasLift_desconocido', 'parkingSpace_False', 'parkingSpace_True',
       'parkingSpace_desconocido', 'price']

In [70]:
df_comprobacion=pd.DataFrame()
df_comprobacion=pd.concat([x_sinout,y], ignore_index=True, axis=1)
df_comprobacion.columns=lista_encabezados

In [74]:
df_comprobacion.tail()

,size,bathrooms,province,municipality,distance,district,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,exterior_False,exterior_True,rooms_0,rooms_1,rooms_2,rooms_3,rooms_4,status_desconocido,status_good,status_newdevelopment,status_renew,floor_1,floor_14,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_bj,floor_desconocido,floor_en,floor_ss,floor_st,hasLift_False,hasLift_True,hasLift_desconocido,parkingSpace_False,parkingSpace_True,parkingSpace_desconocido,price
423,0.30,677.48,693.41,705.59,0.12,687.16,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,640.00
424,0.43,677.48,693.41,690.44,0.51,676.12,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,700.00
425,0.33,677.48,693.41,675.47,0.25,675.47,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,700.00
426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,700.00
427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,695.00


In [82]:
df_comprobacion.dropna(inplace=True)
df_comprobacion.reset_index(inplace=True, drop=True)

In [ ]:
# imputer_iterative= IterativeImputer()       #Aquí se puede poner el KNNImputer o el RandomForestRegressor
#                                             #missing values es por si los nulos no estan como np.nan sino en texto por ejemplo (aunque lo suyo es limpiarlo en el EDA)
#                                             #initial_strategy= "median" sirve para indicar que me lo haga con las medianas
# iterative_imputado=imputer_iterative.fit_transform(df_comprobacion)   #como es solo para numericas se tiene que meter el df de numericas pero hay que definirlo otra vez porque hemos dropeado duplicados

# df_perfect=pd.DataFrame(iterative_imputado, columns= df_comprobacion.columns)  #para que tenga los mismo nombres de columnas
# df_comprobacion[df_comprobacion.columns]= df_perfect    #Con esto metemos todo el dataframe de las numericas sin nulos donde corresponda rellenando esos nulos

In [ ]:
# df_comprobacion.tail()

,size,bathrooms,province,municipality,distance,district,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,exterior_False,exterior_True,rooms_0,rooms_1,rooms_2,rooms_3,rooms_4,status_desconocido,status_good,status_newdevelopment,status_renew,floor_1,floor_14,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_bj,floor_desconocido,floor_en,floor_ss,floor_st,hasLift_False,hasLift_True,hasLift_desconocido,parkingSpace_False,parkingSpace_True,parkingSpace_desconocido,price
423,0.30,677.48,693.41,705.59,0.12,687.16,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,640.00
424,0.43,677.48,693.41,690.44,0.51,676.12,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,700.00
425,0.33,677.48,693.41,675.47,0.25,675.47,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,700.00
426,0.33,672.14,673.14,685.18,0.36,661.32,0.00,0.00,0.04,0.80,0.04,0.11,0.16,0.84,0.12,0.32,0.40,0.16,0.01,0.04,0.93,0.03,0.00,0.22,0.00,0.18,0.13,0.08,0.03,0.01,0.00,0.01,0.15,0.16,0.01,0.00,0.00,0.43,0.52,0.04,0.01,0.29,0.70,700.00
427,0.33,672.19,673.07,685.13,0.36,661.14,0.00,0.00,0.04,0.80,0.04,0.11,0.16,0.84,0.12,0.32,0.40,0.16,0.01,0.04,0.93,0.03,0.00,0.22,0.00,0.18,0.13,0.08,0.03,0.01,0.00,0.01,0.15,0.16,0.01,0.00,0.00,0.43,0.52,0.04,0.01,0.29,0.70,695.00


In [83]:
x_bien=df_comprobacion.drop(columns="price")
y_bien=df_comprobacion["price"]

In [84]:
# Definimos un espacio de parámetros para optimizar
params_rf = {
    'n_estimators': [20],    #número de árboles que queremos que haga
    'max_features': [45],
    'max_depth': [7],
    'min_samples_split': [6],
    'min_samples_leaf': [8], 
    'max_leaf_nodes': [10]
}

modelo_rf= RandomForestRegressor(random_state=42, bootstrap=True)    #bootstrap es para que haga un muestreo con remplazo (hace que dentro de distintos árboles te pueda coger el mismo dato)
grid_search_rf= GridSearchCV(modelo_rf,
                             params_rf,
                             cv=5,
                             scoring= "neg_mean_squared_error",
                             n_jobs=-1)

grid_search_rf.fit(x_bien,y_bien.values)   #esta puesto así porque se necesita un array de una dimension
modelo_final_rf= grid_search_rf.best_estimator_
modelo_final_rf

RandomForestRegressor(max_depth=7, max_features=45, max_leaf_nodes=10,
                      min_samples_leaf=8, min_samples_split=6, n_estimators=20,
                      random_state=42)

In [85]:
y_pred=modelo_final_rf.predict(x_bien)

In [ ]:

metricas = {
'train': {
    'r2_score': r2_score(y_bien, y_pred),
    'MAE': mean_absolute_error(y_bien, y_pred),
    'MSE': mean_squared_error(y_bien, y_pred),
    'RMSE': np.sqrt(mean_squared_error(y_bien, y_pred))
}}

metricas=pd.DataFrame(metricas)
df_metricas_rf= metricas
df_metricas_rf.T

,r2_score,MAE,MSE,RMSE
train,0.77,30.05,"1,632.68",40.41


In [86]:


# calculamos el rmse
rmse = np.sqrt(mean_squared_error(y_bien, y_pred))

print("Precio real vs Predicción:")
display(pd.DataFrame({'Real': y_bien.values, 'Predicción': y_pred}).head(2))
print(f"\nRMSE: {rmse:.2f}")

Precio real vs Predicción:


,Real,Predicción
0,750.00,692.51
1,750.00,697.06



RMSE: 74.55


In [ ]:
# guardo el modelo
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(modelo_final_rf, f)